# Introduction

This notebook is used to collect LANDSAT images and sample them from specified mangrove regions, uploading `.tfrecord` files to Google Cloud in preparation for a machine learning model to work with them.

It's based on part of a notebook prepared by Google as part of the EarthEngine documentation. You can find links to that original notebook below:

<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="http://colab.research.google.com/github/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

# Setup software libraries

Authenticate and import as necessary.

In [0]:
# Cloud authentication.
from google.colab import auth
auth.authenticate_user()

In [0]:
# Import, authenticate and initialize the Earth Engine library.
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=G-ZqCSSE2kk411FzMV3NLWIqm57pC7Kujy8EOql4I3Y&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/zAEoKdRPHlyTzvx1Eojj7mTT4l3jO4CYimKVPw0DotG6nL1rFwn1XeA

Successfully saved authorization token.


In [0]:
# Tensorflow setup.
%tensorflow_version 1.x
import tensorflow as tf

# tf.enable_eager_execution() ## enabled by default
print(tf.__version__)

TensorFlow 1.x selected.
1.15.2


In [0]:
# Folium setup.
import folium
print(folium.__version__)

0.8.3


# Variables

Declare the variables that will be in use throughout the notebook.

## Specify your Cloud Storage Bucket
You must have write access to a bucket to run this demo!  To run it read-only, use the demo bucket below, but note that writes to this bucket will not work.

In [0]:
# INSERT YOUR BUCKET HERE:
BUCKET = 'mangroves_model'

## Set other global variables

In [0]:
# Specify names locations for outputs in Cloud Storage. 
FOLDER = 'fcnn-caribbean-mangroves/data-model-3'
TRAINING_BASE = 'training_patches'
EVAL_BASE = 'eval_patches'

# Specify inputs (Landsat bands) to the model and the response variable.
opticalBands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']
thermalBands = [] ## Not on Lansat 7
BANDS = opticalBands + thermalBands
RESPONSE = 'constant'
FEATURES = BANDS + [RESPONSE]
RESOLUTION = 30

# Specify the size and shape of patches expected by the model.
KERNEL_SIZE = 256
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]
COLUMNS = [
  tf.io.FixedLenFeature(shape=KERNEL_SHAPE, dtype=tf.float32) for k in FEATURES
]
FEATURES_DICT = dict(zip(FEATURES, COLUMNS))

# Specify model training parameters.
BATCH_SIZE = 32
EPOCHS = 100
BUFFER_SIZE = 2000
OPTIMIZER = 'SGD'
LOSS = 'MeanSquaredError'
METRICS = ['RootMeanSquaredError']

YEAR = "2016"

TEST_LOC = [8.527453749657104,-83.29948528935525]
CARIBBEAN_GEO = ee.Geometry.Polygon(
         [[-117, 32],
          [-117, 0],
          [-53, 0],
          [-53, 32]]);

TEST_GEO_SMALL = ee.Geometry.Polygon(
        [[[-83.86332831505052, 9.267295185967846],
          [-83.86332831505052, 8.03455825553625],
          [-82.17692694786302, 8.03455825553625],
          [-82.17692694786302, 9.267295185967846]]]);

# Imagery

Gather and setup the imagery to use for inputs (predictors).  This is a one-year, cloud-free, Landsat 7 composite.  Display it in the notebook for a sanity check.

In [0]:
# Use Landsat 8 surface reflectance data.
l8sr = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR')

# Cloud masking function.
def maskL8sr(image):
  cloudShadowBitMask = ee.Number(2).pow(3).int()
  cloudsBitMask = ee.Number(2).pow(5).int()
  qa = image.select('pixel_qa')
  mask1 = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
    qa.bitwiseAnd(cloudsBitMask).eq(0))
  mask2 = image.mask().reduce('min')
  mask3 = image.select(opticalBands).gt(0).And(
          image.select(opticalBands).lt(10000)).reduce('min')
  mask = mask1.And(mask2).And(mask3)
  return image.select(opticalBands).divide(10000).updateMask(mask)

# The image input data is a cloud-masked median composite.
print("Year is "+str(YEAR))
image = l8sr.filterDate(YEAR+'-01-01', YEAR+'-12-31').map(maskL8sr).median()

# Use folium to visualize the imagery.
mapid = image.getMapId({'bands': ['B3', 'B2', 'B1'], 'min': 0, 'max': 0.3})
map = folium.Map(location=TEST_LOC)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='median composite',
  ).add_to(map)
map

Year is 2016


Prepare the response (what we want to predict).  In this case, we're looking at mangroves. We have to convert the vector FeatureCollection into an Image.

In [0]:
mangrove_shapes = ee.FeatureCollection("users/pandringa/gmw_"+YEAR)

train_geo = ee.FeatureCollection("users/andreagonzales/caribbean_train_areas_"+YEAR);
test_geo = ee.FeatureCollection("users/andreagonzales/caribbean_test_areas_"+YEAR);

polyImage = ee.Image(0).byte().paint(train_geo, 1).paint(test_geo, 2)
polyImage = polyImage.updateMask(polyImage)
print("YEAR is "+YEAR)
mapid = polyImage.getMapId({'min': 1, 'max': 2, 'palette': ['red', 'blue']})
map = folium.Map(location=TEST_LOC, zoom_start=8)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='training polygons',
  ).add_to(map)

mangrove_image = ee.Image(0).byte().paint(mangrove_shapes, 1)
mapid = mangrove_image.updateMask(mangrove_image).getMapId({'min': 0, 'max': 1, 'palette': ['white', 'black']})
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='training polygons',
  ).add_to(map)

map.add_child(folium.LayerControl())
map

YEAR is 2016


Stack the 2D images (Landsat composite and NLCD impervious surface) to create a single image from which samples can be taken.  Convert the image into an array image in which each pixel stores 256x256 patches of pixels for each band.  This is a key step that bears emphasis: to export training patches, convert a multi-band image to [an array image](https://developers.google.com/earth-engine/arrays_array_images#array-images) using [`neighborhoodToArray()`](https://developers.google.com/earth-engine/api_docs#eeimageneighborhoodtoarray), then sample the image at points.

In [0]:
featureStack = ee.Image.cat([
  image.select(BANDS),
  mangrove_image.select(RESPONSE)
]).float()

list = ee.List.repeat(1, KERNEL_SIZE)
lists = ee.List.repeat(list, KERNEL_SIZE)
kernel = ee.Kernel.fixed(KERNEL_SIZE, KERNEL_SIZE, lists)

arrays = featureStack.neighborhoodToArray(kernel)

print(f"Year: {YEAR}")
print(f"Bands: {image.bandNames().getInfo()}")
print(f"Properties: {image.propertyNames().getInfo()}")
print(f"Image projection: {image.projection().getInfo()}")
print(f"Image resolution: {image.projection().nominalScale().getInfo()}")

sample = arrays.sample(
  region = test_geo.first().geometry(), 
  scale = RESOLUTION, 
  numPixels = 32,
  tileScale = 4,
  seed = 2
)
print(f"Sample: {sample.size().getInfo()}")

Year: 2016
Bands: ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']
Properties: ['system:bands', 'system:band_names']
Image projection: {'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [1, 0, 0, 0, 1, 0]}
Image resolution: 111319.49079327357
Sample: 32


# Sampling

Before we can sample the data, we must do a bunch of pre-processing to prepare it. Specifically, we need to estimate the number of samples we want to take from each polygon, based on the area of the polygon versus the total area of all mangrove polygons. (It uses a square root scale function so that it isn't skewed towards the very largest mangrove forests.)

Then, we must also duplicate the polygons that require more than `max_sample` samples, since EarthEngine tends to run into memory issues above that number. By duplicating them and re-setting their sample counts, we effectively sample the correct number from the polygon by visiting it twice (or more) and recording `max_sample` each time.

In [0]:
import math
import random

# Estimated sizes of the training and evaluation datasets.
TRAIN_SIZE = 14000
EVAL_SIZE = 6000

# Maximum number of samples at once (to stay under memory limit)
max_sample = 32


def setSampleCount(areas, target):
  sumRoots = ee.List(areas).map(lambda a: ee.Number(a).sqrt()).reduce(ee.Reducer.sum())
  def mapFn(f):
    f = ee.Feature(f)
    return f.set({
      ## Log-based curve to weight small areas more than large ones
      'samples': ee.Number(f.get('AREA')).sqrt().multiply(target).divide(sumRoots).ceil(),
      'seed': 0
    })
  return mapFn

def repeatPoly(f, samples=None, seed=1):
  f = ee.Feature(f)

  samples = ee.Number(ee.Algorithms.If(samples, samples, f.get('samples')))
  needs_repeat = samples.subtract(max_sample);
  full_repeats = needs_repeat.divide(max_sample).floor();

  f = f.set({ "samples": max_sample, "seed": None })
  coll = ee.List.repeat(f, full_repeats)

  f = f.set({ "samples": needs_repeat.subtract( full_repeats.multiply(max_sample) ) })
  coll = coll.add(f)

  features = ee.FeatureCollection(coll)
  features = features.randomColumn("seed")
  features = features.map(lambda f: f.set({ "seed": ee.Number(f.get("seed")).multiply(1000).int() }))

  return features

def load_and_convert(features_address, samples=10000, name=""):
  print(f"\n[{name}] Processing FeatureCollection...")
  polys = ee.FeatureCollection(features_address)
  count = polys.size().getInfo()
  print(f"[{name}] training count: {count} polys")

  areas = polys.aggregate_array('AREA')
  polys = polys.map( setSampleCount(areas, samples) )

  # Build list of oversized polys, duplicate them into collection
  tooLarge = polys.filter(ee.Filter.gt("samples", max_sample))
  print(f"[{name}] Oversized samples: { tooLarge.size().getInfo() } polys")
  # print(tooLarge.aggregate_array("samples").getInfo())

  repeated = tooLarge.map( repeatPoly ).flatten()
  repeatSize = repeated.size().getInfo()
  print(f"[{name}] New polys added to list: { repeatSize }")
  # print(repeated.aggregate_array("samples").getInfo())
  # print(repeated.aggregate_array("seed").getInfo())

  polys = polys.merge(repeated)

  polys = polys.map(lambda f: f.set({ 'samples': ee.Algorithms.If(ee.Number(f.get('samples')).gt(max_sample), max_sample, f.get('samples')) }))

  # Final counts
  count = polys.size().getInfo()
  polyList = polys.toList(count)

  # Shuffle big ones at the end so they are spread throughout
  for i in range( repeatSize ):
   newPos = random.randint(0, count-repeatSize-1)
   polyList = polyList.swap(count-i-1, newPos)

  totalSamples = polys.reduceColumns(ee.Reducer.sum(), ["samples"]).get('sum').getInfo()
  maxSamples = polys.reduceColumns(ee.Reducer.max(), ["samples"]).get('max').getInfo()
  print(f"[{name}] Adjusted count: { count } polys")
  print(f"[{name}] Estimated samples: { totalSamples }")
  print(f"[{name}] Largest sample: { maxSamples } ")

  print(f"[{name}] Overall average sample per poly: {totalSamples / count }")
  endAvg = polyList.slice(count-repeatSize-1, count).map(lambda f: ee.Feature(f).get("samples")).reduce(ee.Reducer.mean()).getInfo()
  print(f"[{name}] Average sample of last {repeatSize} polys: {endAvg}")

  # Export to EE for debug
  # asset_name = f"{name}_sample_geos_sqrt"
  # print(f"[{name}] Exporting feature collection to EarthEngine asset users/pandringa/{asset_name}")
  # ee.batch.Export.table.toAsset(
  #     collection=polys,
  #     description=asset_name,
  #     assetId="users/pandringa/"+asset_name
  # ).start()

  return polyList, count, totalSamples

print("YEAR is "+YEAR)
trainPolysList, trainCount, trainSampleCount = (load_and_convert(
    "users/andreagonzales/caribbean_train_areas_"+YEAR,
    samples=TRAIN_SIZE,
    name="TRAIN"
))

evalPolysList, evalCount, evalSampleCount = (load_and_convert(
    "users/andreagonzales/caribbean_test_areas_"+YEAR,
    samples=EVAL_SIZE,
    name="EVAL"
))

YEAR is 2016

[TRAIN] Processing FeatureCollection...
[TRAIN] training count: 1278 polys
[TRAIN] Oversized samples: 65 polys
[TRAIN] New polys added to list: 92
[TRAIN] Adjusted count: 1370 polys
[TRAIN] Estimated samples: 14627
[TRAIN] Largest sample: 32 
[TRAIN] Overall average sample per poly: 10.676642335766424
[TRAIN] Average sample of last 92 polys: 10.344086021505376

[EVAL] Processing FeatureCollection...
[EVAL] training count: 536 polys
[EVAL] Oversized samples: 25 polys
[EVAL] New polys added to list: 38
[EVAL] Adjusted count: 574 polys
[EVAL] Estimated samples: 6279
[EVAL] Largest sample: 32 
[EVAL] Overall average sample per poly: 10.939024390243903
[EVAL] Average sample of last 38 polys: 11.564102564102564


Now, we take samples from each polygon and merge the results into a single export. The key step is sampling the array image at points, to get all the pixels in a 256x256 neighborhood at each point. It's worth noting that to build the training and testing data for the FCNN, you export a single TFRecord file that contains patches of pixel values in each record. You do NOT need to export each training/testing patch to a different image. Since each record potentially contains a lot of data (especially with big patches or many input bands), some manual sharding of the computation is necessary to avoid the `computed value too large` error. Specifically, the following code takes multiple (smaller) samples within each geometry, merging the results to get a single export.

Finally, it also uses a little recursion trick to get around the `Maximum depth exceeded` error from EarthEngine which occurred when building up a FeatureCollection using `.merge` in a simple loop. By taking a recursive, split-down-the-middle approach, we reduce the depth of the command issued to EarthEngine from `N` to `log(N)`.

In [0]:
import random
# These numbers determined experimentally.
tileScale = 8
shard_size = 1000 # 1000 samples = 400mb

def merge_samples(geos, start, end):
  if end == start:
    geo = ee.Feature(geos.get(start))
    samples = ee.Number(geo.get('samples'))
    return arrays.sample(
      region = geo.geometry(),
      scale = RESOLUTION,
      numPixels = ee.Algorithms.If(samples.lte(max_sample), samples, max_sample),
      tileScale = tileScale,
      seed = geo.get('seed')
    )
  else:
    pivot = math.floor((end - start) / 2) + start
    return merge_samples(geos, start, pivot).merge( merge_samples(geos, pivot+1, end) )

## Sample usage for visual checking
# ee.batch.Export.table.toAsset(
#     collection = merge_samples(trainPolysList, 0, trainCount-1 ),
#     description = "test_training_sample_full",
#     assetId = "users/pandringa/test_train_area_5th_sample_"+YEAR,
# ).start()

# ee.batch.Export.table.toAsset(
#     collection = merge_samples(evalPolysList, 0, evalCount-1 ),
#     description = "test_eval_sample_full",
#     assetId = "users/pandringa/test_eval_area_5th_sample_"+YEAR,
# ).start()

## Build training data
shards = math.ceil(trainSampleCount / shard_size)
shard_polys = math.ceil(trainCount / shards)
print(f"\nSampling {trainSampleCount} eval samples, in {shards} shards...")
for i in range( shards ):
  shard_start = i * shard_polys
  shard_end = min(shard_start+shard_polys, trainCount)
  geomSample = merge_samples(
      trainPolysList, 
      shard_start, 
      shard_end-1
  )

  desc = TRAINING_BASE + '_' + YEAR +'_s' + str(i)
  task = ee.batch.Export.table.toCloudStorage(
    collection = geomSample,
    description = desc, 
    bucket = BUCKET,
    fileNamePrefix = FOLDER + '/' + desc,
    fileFormat = 'TFRecord',
    selectors = BANDS + [RESPONSE]
  )
  task.start()
  print(f"Started Google Cloud task for {YEAR} training data shard {i}.");

## Build eval data
shards = math.ceil(evalSampleCount / shard_size)
shard_polys = math.ceil(evalCount / shards)
print(f"\nSampling {evalSampleCount} eval samples, in {shards} shards...")
for i in range( shards ):
  shard_start = i * shard_polys
  shard_end = min(shard_start+shard_polys, evalCount)
  geomSample = merge_samples(
      evalPolysList,
      shard_start,
      shard_end-1
  )

  desc = EVAL_BASE + '_' + YEAR +'_s' + str(i)
  task = ee.batch.Export.table.toCloudStorage(
    collection = geomSample,
    description = desc, 
    bucket = BUCKET, 
    fileNamePrefix = FOLDER + '/' + desc,
    fileFormat = 'TFRecord',
    selectors = BANDS + [RESPONSE]
  )
  task.start()
  print(f"Started Google Cloud task for {YEAR} eval data shard {i}.")


Now you're done! Check the [Earth Engine Console](https://code.earthengine.google.com) to view the status of all your tasks, and once they're done you can move onto the other notebooks that handle data loading and model training.